In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam
from keras.utils import to_categorical



/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/entilza/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 100, 100
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 4)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 5
# Количество изображений для обучения
nb_train_samples = 224
# Количество изображений для проверки
nb_validation_samples = 28
# Количество изображений для тестирования
nb_test_samples = 28

In [3]:
classes = ['круассан','пончик', 'полки','завиток']

In [4]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
vgg16_net.trainable = False
#vgg16_net.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
model = Sequential()
#model.add(Conv2D(32, (3, 3), input_shape=input_shape))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(32, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

In [7]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 224 images belonging to 4 classes.


In [9]:
# читаем данные из файловой системы
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = False,
    class_mode='categorical')

Found 28 images belonging to 4 classes.


In [10]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 28 images belonging to 4 classes.


In [11]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples// batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30


ValueError: A target array with shape (5, 4) was passed for an output of shape (None, 3) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block3_conv1':
        trainable = True
    layer.trainable = trainable

In [ ]:
#model.compile(loss='categorical_crossentropy',
#              optimizer=Adam(lr=1e-5), 
#              metrics=['accuracy'])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [ModelCheckpoint('saveModelWeightStep/mnist-dense-{epoch:02d}-{val_acc:.4f}.hdf5')]


In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
#model.save('epoch=5,acc_test=89,89.h5')

In [ ]:
# создаем csv-файл со списком тестовых картинок и отображаем результаты теста по каждой
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas
spisoktest = pandas.read_csv("./test.csv")
for k in range(len(spisoktest)):
    print(spisoktest.iloc[k,0])
    im = spisoktest.iloc[k,0]
    img = image.load_img(im, target_size=(img_width, img_height))
    # Преобразуем картинку в массив
    x = image.img_to_array(img)
    x = x.reshape(1, x.shape[0],x.shape[1],x.shape[2])
    # Нормализуем изображение
    x /= 255
    prediction = model.predict(x)
    prediction
    prediction = np.argmax(prediction)
    print("Номер класса:", prediction)
    print("Название класса:", classes[prediction])
        

In [ ]:
#ПРОВЕРКА НА ОДНОЙ КАРТИНКЕ
imgM = image.load_img('download.jpg', target_size=(100, 100))
#pic = pandas.read("ponchik.64.jpg")
# Нормализуем изображение
plt.imshow(imgM)
plt.show()

x = image.img_to_array(imgM)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

#img /= 255
prediction = model.predict(x)
#prediction.shape
prediction = np.argmax(prediction)
print("Номер класса:", prediction)
print("Название класса:", classes[prediction])

In [ ]:
import cv2
imgM = cv2.imread('Screenshot from magnitPonchik.png')
#imgM = image.load_img('Screenshot from magnitPonchik.png')
print(imgM.shape)

In [ ]:
#heigh = imgM.shape[1]//5
#width = imgM.shape[0]//3
heigh = 100
width = 100

maskShape = (heigh, width, 3)
if len(imgM)==2:
    maskShape = (heigh, width)
print(heigh, width)
mask = np.zeros(maskShape, dtype = np.uint8)
#mask = imgM[0:heigh, 0:width]
#mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
print(mask.shape)

In [ ]:
listX = [0]
w=0
while w < imgM.shape[1]:
    w+=width
    if w>imgM.shape[1]:
        w=imgM.shape[1]
    listX.append(w)
print(listX)

listY = [0]
h=0
while h < imgM.shape[0]:
    h+=heigh
    if h>imgM.shape[0]:
        h=imgM.shape[0]
    listY.append(h)
print(listY) 

In [ ]:
#print(mask.shape)
for x in range(len(listX)):
    for y in range(len(listY)):
        
        if listY[y]==listY[-1] or listX[x]==listX[-1]:
            break
        #print(listY[y], listY[y+1], listX[x], listX[x+1])
        #print(type(mask))
        #print(type(imgM))
        dy = listY[y+1]-listY[y]
        dx = listX[x+1]-listX[x]
        mask [:dy,:dx]= imgM[listY[y]:listY[y+1], listX[x]:listX[x+1]]
        
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        plt.imshow(mask)
        plt.show()
        pic = image.img_to_array(mask)
        pic = np.expand_dims(pic, axis=0)
        #mask= preprocess_input(mask)

        pic /= 255
        prediction = model.predict(pic)
        prediction.shape
        prediction = np.argmax(prediction)
        print("Номер класса:", prediction)
        print("Название класса:", classes[prediction])

In [ ]:
#h = 0
#w = 0
#while h<=imgM.shape[0] and w <= imgM.shape[1]:
#    Y=h
#    X=w
#    h += heigh
#    w += width
#    if h>imgM.shape[0]:
#        h = imgM.shape[0]
#        print('yve')
#    print(Y, X)
#    print(h, w, imgM.shape[1])
    
#    mask = imgM[Y:h, X:w]
#    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
#    print(mask.shape)
#    plt.imshow(mask)
3    plt.show()
    

In [ ]:
#print(mask.shape)

In [ ]:
#plt.imshow(mask)
#plt.show()